In [6]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

### Library

In [7]:
from datetime import datetime, timedelta
import oandapy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import pickle
import sys
import os
import talib

In [8]:
sys.path.append('/Users/toshio/MyProject/fx')
from config import token
from lib.indicator import ichimoku

### 過去データの読み込み

In [9]:
gran = 'H1'

In [10]:
with open('../intermediate_data/data_{}.pickle'.format(gran), mode='rb') as f:
    df = pickle.load(f)

### データをohlcvの形に変換

In [11]:
df_ask = pd.DataFrame(columns = ['open', 'high', 'low', 'close', 'volume'])
df_ask['open'] = df['openAsk']
df_ask['high'] = df['highAsk']
df_ask['low'] = df['lowAsk']
df_ask['close'] = df['closeAsk']
df_ask['volume'] = df['volume']
arr_ask = np.array(df_ask)

In [12]:
df_bid = pd.DataFrame(columns = ['open', 'high', 'low', 'close', 'volume'])
df_bid['open'] = df['openBid']
df_bid['high'] = df['highBid']
df_bid['low'] = df['lowBid']
df_bid['close'] = df['closeBid']
df_bid['volume'] = df['volume']
arr_bid = np.array(df_bid)

### 前日終値との差を取得
翌日の終値がどれだけ下がるかを各日付の値とする

In [13]:
delta = pd.DataFrame(index = df.index[1:], columns = ['delta_close'])
delta['delta_close'] = df_ask['close'].diff()[:-1]

### テクニカル指標の算出

In [14]:
sma = pd.DataFrame(index = df.index, columns = ['sma5', 'sma25', 'sma50', 'sma75'])
sma['sma5'] = talib.SMA(arr_ask[:,3], timeperiod = 5)
sma['sma25'] = talib.SMA(arr_ask[:,3], timeperiod = 25)
sma['sma50'] = talib.SMA(arr_ask[:,3], timeperiod = 50)
sma['sma75'] = talib.SMA(arr_ask[:,3], timeperiod = 75)

In [15]:
macd = pd.DataFrame(index = df.index, columns = ['macd', 'macdsignal', 'macdhist'])
macd['macd'] =  talib.MACD(arr_ask[:,3],fastperiod=12, slowperiod=26, signalperiod=9)[0]
macd['macdsignal'] =  talib.MACD(arr_ask[:,3],fastperiod=12, slowperiod=26, signalperiod=9)[1]
macd['macdhist'] =  talib.MACD(arr_ask[:,3],fastperiod=12, slowperiod=26, signalperiod=9)[2]

In [16]:
rsi = pd.DataFrame(index = df.index, columns = ['rsi'])
rsi['rsi'] =  talib.RSI(arr_ask[:,3], timeperiod = 14)

In [17]:
bband = pd.DataFrame(index = df.index, columns = ['-3sigma', '-2sigma', '-1sigma', '+1sigma', '+2sigma', '+3sigma'])
bband['+1sigma'] = talib.BBANDS(arr_ask[:,3], timeperiod=15, nbdevup=1, nbdevdn=1)[0]
bband['-1sigma'] = talib.BBANDS(arr_ask[:,3], timeperiod=15, nbdevup=1, nbdevdn=1)[2]
bband['+2sigma'] = talib.BBANDS(arr_ask[:,3], timeperiod=15, nbdevup=2, nbdevdn=2)[0]
bband['-2sigma'] = talib.BBANDS(arr_ask[:,3], timeperiod=15, nbdevup=2, nbdevdn=2)[2]
bband['+3sigma'] = talib.BBANDS(arr_ask[:,3], timeperiod=15, nbdevup=3, nbdevdn=3)[0]
bband['-3sigma'] = talib.BBANDS(arr_ask[:,3], timeperiod=15, nbdevup=3, nbdevdn=3)[2]

In [18]:
adx = pd.DataFrame(index = df.index, columns = ['adx'])
adx['adx'] = talib.ADX(arr_ask[:,1], arr_ask[:,2], arr_ask[:,3], timeperiod =14)

In [19]:
di = pd.DataFrame(index = df.index, columns = ['+di', '-di'])
di['+di'] = talib.PLUS_DI(arr_ask[:,1], arr_ask[:,2], arr_ask[:,3], timeperiod = 14)
di['-di'] = talib.MINUS_DI(arr_ask[:,1], arr_ask[:,2], arr_ask[:,3], timeperiod = 14)

In [20]:
sar = pd.DataFrame(index = df.index, columns = ['sar'])
sar['sar'] = talib.SAR(arr_ask[:,1],arr_ask[:,2], acceleration=0.05, maximum=0.2)

In [21]:
ichi = ichimoku(df_ask).drop('close', axis = 1)

### テクニカル指標を結合

In [22]:
adder = [delta, sma, macd, rsi, bband, adx, di, sar, ichi]
data = df.join(adder)
data = data.dropna()

### 出力

In [23]:
with open('../intermediate_data/prep_data_{}.pickle'.format(gran), mode='wb') as f:
    pickle.dump(data, f)